In [0]:
%pip install openpyxl

In [0]:
%restart_python

In [0]:
%pip list | grep openpyxl 

In [0]:
import pandas as pd

# Read Excel with pandas and save as CSV
pdf = pd.read_excel("/Volumes/pyspark_catalog/source/db_volume/netflix_data/NFLX_top10.xlsx")
pdf.to_csv("/Volumes/pyspark_catalog/source/db_volume/netflix_data/NFLX_top10.csv", index=False)

In [0]:
pdf = pd.read_excel("/Volumes/pyspark_catalog/source/db_volume/netflix_data/IMDB_rating1.xlsx")
pdf.to_csv("/Volumes/pyspark_catalog/source/db_volume/netflix_data/IMDB_rating.csv", index=False)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
#Step 1: Load the datasets from Excel files
# Load NFLX Top 10 sheet
top10_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("treatEmptyValuesAsNulls", "true") \
    .load("/Volumes/pyspark_catalog/source/db_volume/netflix_data/NFLX_top10.csv")

In [0]:
display(top10_df)

In [0]:
# Load IMDB Rating sheet (contains title, rating)
imdb_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("treatEmptyValuesAsNulls", "true") \
    .load("/Volumes/pyspark_catalog/source/db_volume/netflix_data/IMDB_rating.csv")

In [0]:
display(imdb_df)

In [0]:
#Step 2: Data Cleaning and Joining

joined_df = top10_df.join(imdb_df.withColumnRenamed("title", "show_title"), on="show_title", how="inner") \
    .filter(col("rating").isNotNull() & (col("rating") > 0))
joined_df.show(10)

In [0]:
# Step 3: Analysis 1 - IMDb Rating and Weekly Rank
# Compute average weekly rank by rating buckets (e.g., <5, 5-7, >7)
# Create rating buckets
# Lower weekly_rank number means better rank (1 is top)
joined_df_with_buckets = joined_df.withColumn("rating_bucket",
    when(col("rating") < 5, "<5")
    .when((col("rating") >= 5) & (col("rating") <= 7), "5-7")
    .otherwise(">7"))
display(joined_df_with_buckets)

In [0]:
# Average weekly rank per bucket (lower avg rank is better)
avg_rank_by_bucket = joined_df_with_buckets.groupBy("rating_bucket") \
    .agg(round(avg("weekly_rank"), 2).alias("avg_weekly_rank")) \
    .orderBy("rating_bucket")

avg_rank_by_bucket.show()

In [0]:
# Examples of low-rated but high-ranked titles (weekly_rank <=5, rating <5)
low_rating_high_rank_examples = joined_df.filter((col("weekly_rank") <= 5) & (col("rating") < 5)) \
    .select("show_title", "rating", "weekly_rank") \
    .distinct() \
    .orderBy("weekly_rank")

low_rating_high_rank_examples.show(10)

In [0]:
# Examples of high-rated but low-ranked titles (rating >7, weekly_rank >5)
high_rating_low_rank_examples = joined_df.filter((col("rating") > 7) & (col("weekly_rank") > 5)) \
    .select("show_title", "rating", "weekly_rank") \
    .distinct() \
    .orderBy(desc("weekly_rank"))

high_rating_low_rank_examples.show(10)

In [0]:
# Correlation between rating and weekly_rank (negative correlation would mean higher rating -> better rank)
correlation_rank = joined_df.stat.corr("rating", "weekly_rank")
print(f"Correlation between IMDb Rating and Weekly Rank: {correlation_rank}")

In [0]:
# Step 4: Analysis 2 - IMDb Ratings and Cumulative Weeks in Top 10
# Insight: Higher-rated titles (>7) tend to stay longer in Top 10.

# Average cumulative weeks by rating bucket
avg_cumulative_by_bucket = joined_df_with_buckets.groupBy("rating_bucket") \
    .agg(round(avg("cumulative_weeks_in_top_10"), 2).alias("avg_cumulative_weeks")) \
    .orderBy("rating_bucket")

avg_cumulative_by_bucket.show()

In [0]:
# Titles with high rating (>7) and high cumulative weeks (>5)
high_rating_long_stay = joined_df.filter((col("rating") > 7) & (col("cumulative_weeks_in_top_10") > 5)) \
    .select("show_title", "rating", "cumulative_weeks_in_top_10") \
    .distinct() \
    .dropDuplicates(["show_title"])\
    .orderBy(desc("cumulative_weeks_in_top_10"))

high_rating_long_stay.show(10)

In [0]:
# Correlation between rating and cumulative_weeks_in_top_10
correlation_cumulative = joined_df.stat.corr("rating", "cumulative_weeks_in_top_10")
print(f"Correlation between IMDb Rating and Cumulative Weeks in Top 10: {correlation_cumulative}")

In [0]:
# Step 5: Analysis 3 - IMDb Ratings and Weekly Hours Viewed
# Insight: Relationship is inconsistent; some low-rated titles have high hours viewed due to curiosity/marketing.

# Average weekly hours viewed by rating bucket
avg_hours_by_bucket = joined_df_with_buckets.groupBy("rating_bucket") \
    .agg(round(avg("weekly_hours_viewed"), 0).alias("avg_weekly_hours_viewed")) \
    .orderBy("rating_bucket")

avg_hours_by_bucket.show()

In [0]:
# Total hours viewed per title, grouped by rating bucket
total_hours_by_bucket = joined_df_with_buckets.groupBy("rating_bucket") \
    .agg(sum("weekly_hours_viewed").alias("total_weekly_hours_viewed")) \
    .orderBy("rating_bucket")

total_hours_by_bucket.show()

In [0]:
# Examples of low-rated but high hours viewed (rating <5, weekly_hours_viewed > 10M)
low_rating_high_hours = joined_df.filter((col("rating") < 5) & (col("weekly_hours_viewed") > 10000000)) \
    .select("show_title", "rating", "weekly_hours_viewed") \
    .distinct() \
    .orderBy(desc("weekly_hours_viewed"))

low_rating_high_hours.show(10)

In [0]:
# Correlation between rating and weekly_hours_viewed
correlation_hours = joined_df.stat.corr("rating", "weekly_hours_viewed")
print(f"Correlation between IMDb Rating and Weekly Hours Viewed: {correlation_hours}")

In [0]:
# Step 6: Overall Insights Summary (Printed for Reference)
print("""
Insights:
1. IMDb Rating and Weekly Rank: High ratings don't guarantee top ranks. Correlation is {corr_rank}. Examples show low-rated titles like '365 Days: This Day' (2.5) ranking high.
2. IMDb Ratings and Cumulative Weeks: Higher ratings correlate with longer stays (corr: {corr_cum}). Titles like 'Yo soy Betty, la fea' (>8 rating) stay multiple weeks.
3. IMDb Ratings and Weekly Hours Viewed: Inconsistent; low-rated titles can have high hours (corr: {corr_hours}). Marketing drives short-term views.""")